In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import netCDF4 as nc
import xarray as xr
import scipy.ndimage
import scipy as sp

In [20]:
# Read in netcdf data

ds = nc.Dataset('/Users/clara.henry/Documents/Derwael/Code/DerwaelBC.nc')
bedrock_old = np.array(ds.variables['bedrock'][:])
upper_surface_old = np.array(ds.variables['zs0'][:])
lower_surface_old = np.array(ds.variables['zb0'][:])
X = np.array(ds.variables['x'][:])
Y = np.array(ds.variables['y'][:])


In [21]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    CDI: Climate Data Interface version 2.0.5 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.6
    history: Wed Aug 10 15:10:00 2022: cdo chname,zs,zs0 Derwael0.nc DerwaelBC.nc
Wed Aug 10 15:10:00 2022: cdo chname,zb,zb0 Derwael.nc Derwael0.nc
Wed Aug 10 15:02:47 2022: cdo merge DerwaelNewBedrock2.nc Derwael_surface2.nc Derwael_thickness.nc Derwael_base.nc Derwael_bed.nc Derwael_mask.nc Derwael_vx2.nc Derwael_vy2.nc Derwael.nc
Wed Aug 10 15:02:45 2022: cdo chname,__xarray_dataarray_variable__,bedrock DerwaelNewBedrock.nc DerwaelNewBedrock2.nc
    CDO: Climate Data Operators version 2.0.5 (https://mpimet.mpg.de/cdo)
    dimensions(sizes): x(156), y(173), x_2(174), y_2(193)
    variables(dimensions): float64 x(x), float64 y(y), float64 x_2(x_2), float64 y_2(y_2), float32 bedrock(y, x), float32 zs0(y, x), float32 thickness(y, x), float32 zb0(y, x), float32 bed(y, x), float32 mask(y, x), float32 vx(y_2

In [22]:
count = 0

for x in X:
    if x < 998000:
        count += 1

count

155

In [23]:
bedrock_old[0][1]

-334.93427

In [24]:
for i in range(40):
    for j in range(len(Y)):
        if bedrock_old[j][i] > -300:
            bedrock_old[j][i] = -300

In [25]:
for i in range(140,156):
    for j in range(len(Y)):
        if upper_surface_old[j][i] < 20:
            upper_surface_old[j][i] = 20
            lower_surface_old[j][i] = -180


In [26]:
bedrock_new = bedrock_old #.transpose()
upper_surface_new = upper_surface_old
lower_surface_new = lower_surface_old

da_bed = xr.DataArray(
    data=bedrock_new,
    dims=['y', 'x'],
    coords=dict(
        x=X,
        y=Y
    
    )
)

da_upp = xr.DataArray(
    data=upper_surface_new,
    dims=['y', 'x'],
    coords=dict(
        x=X,
        y=Y
    
    )
)

da_low = xr.DataArray(
    data=lower_surface_new,
    dims=['y', 'x'],
    coords=dict(
        x=X,
        y=Y
    
    )
)


da_bed.to_netcdf('DerwaelNewBedrock.nc')
da_upp.to_netcdf('DerwaelUpper.nc')
da_low.to_netcdf('DerwaelLower.nc')
da_bed.close()
da_upp.close()
da_low.close()
da_bed


<xarray.DataArray (y: 173, x: 156)>
array([[-334.2401 , -334.93427, -335.97546, ..., -546.8216 , -542.83026,
        -538.6654 ],
       [-333.02536, -333.19888, -333.89304, ..., -546.9951 , -542.83026,
        -539.35956],
       [-330.42233, -331.11646, -331.46353, ..., -546.648  , -542.83026,
        -539.8801 ],
       ...,
       [-431.24667, -432.11435, -433.3291 , ..., -371.8974 , -371.20325,
        -369.9885 ],
       [-432.11435, -433.3291 , -434.89093, ..., -372.418  , -372.07092,
        -371.20325],
       [-433.3291 , -435.41153, -436.62628, ..., -373.63275, -372.9386 ,
        -372.59155]], dtype=float32)
Coordinates:
  * x        (x) float64 9.205e+05 9.21e+05 9.215e+05 ... 9.975e+05 9.98e+05
  * y        (y) float64 1.966e+06 1.966e+06 1.965e+06 ... 1.88e+06 1.88e+06